In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from urllib.request import urlopen
import requests
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options




options = webdriver.FirefoxOptions()
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--headless")
driver = webdriver.Firefox( options=options)
driver.maximize_window() # For maximizing window
driver.implicitly_wait(20)


# Add the browser url below after selecting city and dates.
# example link: https://www.makemytrip.com/hotels/hotel-listing/?checkin=08212023&city=CTCCU&checkout=08222023&roomStayQualifier=2e0e&locusId=CTCCU&country=IN&locusType=city&searchText=Kolkata&regionNearByExp=3&rsc=1e2e0e

MMT_LINK = "https://www.makemytrip.com/hotels/hotel-listing/?checkin=02112024&checkout=02122024&locusId=CTCTH&locusType=city&city=CTCTH&country=VNM&searchText=Cat%20Ba&roomStayQualifier=2e0e&_uCurrency=USD&mmAreaTag=Cat%20Ba%7CAR16508081373642777&reference=hotel&filterData=HOTEL_CATEGORY%7CNear%20Indian%20Restaurants&type=area&rsc=1e2e0e"

# Add the path were the CSV file is present(below), so that the scraped data can be saved.
# Note: First create the CSV file based on the sample dataset given before running the script, so that you wont fall into any trouble!

CSV_PATH = "hotelcatba.csv"

driver.get(MMT_LINK)
time.sleep(6)
print("6 sec over")




for i in range(0,3300):
    print("hotel: "+str(i))

    content = driver.find_element(By.XPATH,'//*[@id="Listing_hotel_'+str(i)+'"]')
    hname = content.find_element(By.ID,'hlistpg_hotel_name')
    print(hname.text)
    try:
        rating = content.find_element(By.XPATH,'//*[@id="Listing_hotel_'+str(i)+'"]/a/div/div/div[2]/div[1]/div/span/span[2]/span')
        rating = rating.text
        print(rating)

        review_count = content.find_element(By.ID,'hlistpg_hotel_reviews_count')
        review_count = review_count.text
        print(review_count)
    except:
        rating=""
        #rating_desc=""
        review_count=""

    loc = content.find_element(By.CLASS_NAME,'pc__html')
    loc = loc.text
    loc = loc.split("|")
    location = loc[0] #hotel_locationzdb
    try:
        landmark = loc[1].split('from')
        dist_landmark = landmark[0].lstrip() #distance to nearest landmark/locality
        landmark = landmark[1].lstrip() #nearest landmark/locality
    except:
        dist_landmark=""
        landmark=""

    print("location: "+location)
    print("landmark: "+landmark)
    print("dis to landmark: "+dist_landmark)


    price = content.find_element(By.ID,'hlistpg_hotel_shown_price')
    print(price.text[2:])

    tax = content.find_element(By.XPATH,'//*[@id="Listing_hotel_'+str(i)+'"]/a/div[1]/div/div[2]/div/div/p[2]')
    try:
        tax = tax.text.split(" ")[2]
    except:
        tax=""
    print(tax)

    try:
        s_rating = content.find_element(By.ID,'hlistpg_hotel_star_rating')
        s_rating = s_rating.get_attribute('data-content')
    except:
        s_rating=""

    print("s_rating: "+s_rating) #star_rating



    #csv
    data=[[hname.text,rating,review_count,s_rating,location,landmark,dist_landmark,price.text[2:],tax]]
    with open(CSV_PATH,'a',newline='') as file:
                writer=csv.writer(file)
                writer.writerows(data)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

driver.close()
